<a href="https://colab.research.google.com/github/wcruz-br/imersao-ia-alura-google/blob/main/projeto/Mentor_de_habitos_saudaveis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### BEM VINDO AO MENTOR DE HÁBITOS SAUDÁVEIS

Esse projeto lhe fará algumas perguntas e, com base nelas, fará recomendações de hábitos saudáveis. As informações são baseadas no [Guia de Atividade Física para a População Brasileira](https://bvsms.saude.gov.br/bvs/publicacoes/guia_atividade_fisica_populacao_brasileira.pdf), do Ministério da Saúde (2021).

## Inicialização

Este bloco inicializa a IA e carrega as informações necessárias para seu funcionamento.

In [1]:
# !pip install google-generativeai PyPDF2 requests

import requests
from PyPDF2 import PdfReader
import google.generativeai as genai
from google.colab import userdata
import numpy as np
import pandas as pd

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

# Configura os documentos do embed
documents = [
  {"titulo": "Tenha mais saúde e qualidade de vida com hábitos saudáveis",
   "conteudo": "A atividade física deve ser praticada em todas as fases da vida e em diversos momentos, como ao se deslocar de um lugar para outro, durante o trabalho ou estudo, ao realizar tarefas domésticas ou durante o tempo livre. Os exercícios físicos também são exemplos de atividades físicas, mas se diferenciam por serem atividades planejadas, estruturadas e repetitivas com o objetivo de melhorar ou manter as capacidades físicas e o peso adequado, além de serem prescritos por profissionais de educação física. Todo exercício físico é uma atividade física, mas nem toda atividade física é um exercício físico.\nAlguns dos benefícios para sua saúde são: o controle do peso; a diminuição da chance de desenvolvimento de cânceres e de doenças crônicas, como a diabetes, pressão alta e doenças do coração; a melhora da disposição; e a promoção da interação social."},
  {"titulo": "exemplos de atividade física",
   "conteudo": "caminhar, correr, pedalar, brincar, subir escadas, carregar objetos, dançar, limpar a casa, passear com animais de estimação, cultivar a terra, cuidar do quintal, praticar esportes, lutas, ginásticas, yoga, entre outros"},
  {"titulo": "benefícios da atividade física",
   "conteudo": "Promove o seu desenvolvimento humano e bem-estar, ajudando a desfrutar de uma vida plena com melhor qualidade; \nPrevine e diminui a mortalidade por diversas doenças crônicas, tais como pressão alta, diabetes (alto nível de açúcar no sangue), doenças do coração e alguns tipos de câncer; \nAjuda a controlar o seu peso, melhorando não apenas a saúde, mas também a relação com seu corpo; \nDiminui os sintomas da asma; \nDiminui o uso de medicamentos em geral; \nDiminui o estresse e sintomas de ansiedade e depressão; \nMelhora o seu sono; \nPromove prazer, relaxamento, divertimento e disposição."},
  {"titulo": "quando fazer atividade física",
   "conteudo": "no seu tempo livre; quando você se desloca; nas atividades do trabalho ou dos estudos; e nas tarefas domésticas"},
  {"titulo": "atividade física no tempo livre",
   "conteudo": "A atividade física no tempo livre é feita no seu tempo disponível ou no lazer, baseada em preferências e oportunidades. Você pode caminhar, correr, empinar pipa, dançar, nadar, fazer trilha, pedalar, surfar, pular corda, jogar futebol, vôlei, basquete, bocha, tênis, peteca, taco/bete, frescobol, praticar ginástica, musculação, hidroginástica, artes marciais, capoeira, yoga, ou participar de brincadeiras e jogos, como esconde-esconde, pega-pega, saltar elástico, queimada/baleado/carimba/caçador, entre outras"},
  {"titulo": "atividade física no deslocamento",
   "conteudo": "A atividade física no deslocamento é feita como forma de deslocamento ativo para ir de um lugar a outro. Você pode caminhar, manejar a cadeira de rodas, pedalar, remar, patinar, andar a cavalo, de skate ou de patinete (sem motor), entre outras."},
  {"titulo": "atividade física no trabalho ou estudo",
   "conteudo": "Tornar seu dia a dia mais ativo também é uma forma de atividade física. Nesse caso, você pode optar por subir escadas ao invés de usar o elevador. Se o local em que você trabalha ou estuda oferece atividade física, participe de alguma ou procure locais próximos onde seja possível praticar atividade física"},
  {"titulo": "atividade física nas tarefas domésticas",
   "conteudo": "A atividade física nas tarefas domésticas é feita para o cuidado do lar e da família. Você pode cuidar das plantas, cortar a grama, fazer compras, dar banho na criança, no idoso, na pessoa que requer cuidados ou no animal de estimação, varrer, esfregar ou lavar, entre outras."},
  {"titulo": "Intensidades da atividade física",
   "conteudo": "A intensidade é o grau do esforço físico necessário para fazer uma atividade física. Normalmente, quanto maior a intensidade, maior é o aumento dos batimentos do coração, da respiração, do gasto de energia e da percepção de esforço.\nPara saber qual a intensidade da atividade física que você está praticando, preste atenção em como você se sente. A intensidade pode ser:\nLeve: exige mínimo esforço físico e causa pequeno aumento da respiração e dos batimentos do seu coração. Numa escala de 0 a 10, a percepção de esforço é de 1 a 4. Você vai conseguir respirar tranquilamente e conversar normalmente enquanto se movimenta ou até mesmo cantar uma música.\nModerada: exige mais esforço físico, faz você respirar mais rápido que o normal e aumenta moderadamente os batimentos do seu coração. Numa escala de 0 a 10, a percepção de esforço é 5 e 6.Você vai conseguir conversar com dificuldade enquanto se movimenta e não vai conseguir cantar.\nVigorosa: exige um grande esforço físico, faz você respirar muito mais rápido que o normal e aumenta muito os batimentos do seu coração. Numa escala de 0 a 10, a percepção de esforço é 7 e 8. Você não vai conseguir nem conversar enquanto se movimenta."},
  {"titulo": "diferença entre atividade física e exercício físico",
   "conteudo": "Todo exercício físico é uma atividade física, mas nem toda atividade física é um exercício físico. Ou seja, o exercício físico é um tipo de atividade física planejada, estruturada e repetitiva que tem o objetivo de melhorar ou manter as capacidades físicas e o peso adequado."},
  {"titulo": "comportamento sedentário",
   "conteudo": "Comportamento sedentário envolve atividades realizadas quando você está acordado sentado, reclinado ou deitado e gastando pouca energia. Por exemplo, quando você está em uma dessas posições para usar celular, computador, tablet, videogame e assistir à televisão ou à aula, realizar trabalhos manuais, jogar cartas ou jogos de mesa, dentro do carro, ônibus ou metrô.\nEvite ficar em comportamento sedentário. Sempre que possível, reduza o tempo em que você permanece sentado ou deitado assistindo à televisão ou usando o celular, computador, tablet ou videogame. Por exemplo, a cada uma hora, movimente-se por pelo menos 5 minutos e aproveite para mudar de posição e ficar em pé, ir ao banheiro, beber água e alongar o corpo. São pequenas atitudes que podem ajudar a diminuir o seu comportamento sedentário e melhorar sua qualidade de vida."},
  {"titulo": "praticar atividade física sozinho ou com companhia",
   "conteudo": "Você pode praticar atividade física sozinho ou com companhia, faça como preferir. A maioria dos esportes e jogos é feita com companhia. Aproveite para chamar os amigos, familiares, vizinhos ou seus colegas de trabalho.\nOs momentos da família reunida também podem ser uma oportunidade para fazer atividade física, como praticar esportes, passear com o animal de estimação, passear de bicicleta e fazer atividades na natureza. Ou nas tarefas domésticas, tendo a ajuda uns dos outros, seja lavando a louça, retirando o lixo, contribuindo com a limpeza ou cozinhando. \nOs colegas de trabalho também podem ser uma boa companhia para aumentar a atividade física no deslocamento ativo, no próprio trabalho ou no tempo livre. Uma caminhada é sempre bem-vinda para ser ativo no seu dia."},
  {"titulo": "cuidados ao praticar atividade física",
   "conteudo": "Para a prática de atividade física, se possível, use roupas leves e calçados confortáveis.\nNas atividades físicas ao ar livre, utilize alguma forma de proteção, como boné, camisa de manga longa e protetor solar, se possível.\nVocê deve beber água antes, durante e após a prática de atividade física. Também deve ter uma alimentação adequada e saudável sempre que possível.\nSe você tiver alguma lesão ou sentir algum desconforto anormal, como alguma dor na região do peito ou tontura, procure a Unidade Básica de Saúde mais próxima de você.\nFique atento se você tem diabetes (alto nível de açúcar no sangue), especialmente quando praticar atividade física mais vigorosa. Para evitar hiperglicemia (níveis elevados de açúcar no sangue) ou hipoglicemia (níveis baixos de açúcar no sangue), você precisará ajustar a sua dose de insulina e a sua alimentação nos períodos em que for fazer atividade física. Procure um profissional de saúde para ajudar você.\nSe você tem asma, fique atento às situações que podem desencadear alergias, como o clima seco e o cloro da piscina. O uso de medicação para asma, aliado ao tipo, local e horário escolhidos para fazer a atividade física, ajuda a aliviar os sintomas.\nVocê deve lembrar que o sono e a alimentação adequada e saudável também são importantes para a prática de atividade física.\nSe você tiver dúvidas sobre a prática de atividade física e as doenças crônicas, procure a Unidade Básica de Saúde mais próxima de você.\nRespeite seus limites!"}
]

# Coloca num dataframe
df = pd.DataFrame(documents)
df.columns = ['Titulo', 'Conteudo']

# Obtém o embedding dos documentos e acrescenta ao dataframe
model = 'models/embedding-001'
def embed_fn(title, text):
  return genai.embed_content(model=model,
                            content=text,
                            task_type="RETRIEVAL_DOCUMENT",
                            title=title)["embedding"]
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)

# função apra buscar a resposta mais adequada
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                              content=consulta,
                                              task_type="RETRIEVAL_QUERY")["embedding"]
  # traça a "distância" entre o embed da consulta e o embed de cada um dos textos
  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)
  # retorna a maior similaridade entre o texto e a consulta
  #indice = np.argmax(produtos_escalares)
  #return df.iloc[indice]["Conteudo"]
  # retorna os dois mais similares
  n_embeddings = 2
  indices = np.argpartition(produtos_escalares, -n_embeddings)[-n_embeddings:]  # Top N indices
  top_contents = []
  for index in indices:
      top_contents.append(df.iloc[index]["Conteudo"])
  return "\n".join(top_contents)

# Prepara o modelo generativo que vai trabalhar a resposta obtida
config_de_geracao = {
  "temperature": 0.5,
  "max_output_tokens": 2000,
}
config_de_seguranca = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_LOW_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_LOW_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_LOW_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_LOW_AND_ABOVE"
  }
]
model_2 = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                                generation_config=config_de_geracao,
                                safety_settings=config_de_seguranca)

# Conversa

Vamos conversar sobre hábitos saudáveis?

In [ ]:
mensagem = 'Olá! Vamos falar sobre hábitos saudáveis? Comece me contando se você tem se exercitado e se deseja fazer mais atividade física.\nQuando quiser encerrar nossa conversa, digite "fim".\n\n'
while True:
    consulta = input(mensagem)
    if consulta.strip().lower().endswith('fim'):
        break
    trecho = gerar_e_buscar_consulta(consulta, df, model)
    #print(trecho)
    prompt = f"Reescreva esse texto de maneira simpática e incentivadora, respondendo a uma pessoa adulta a pergunta '{consulta}' mas sem adicionar informações que não façam parte do texto: {trecho}"
    resposta = model_2.generate_content(prompt)
    print("\n" + resposta.text)
    mensagem = "\nQuer saber mais? É só perguntar!\n"

print("\n\nObrigado e até a próxima!")